In [33]:
import os
import re
import time
import datetime
import requests
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
from IPython.display import clear_output
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [36]:
player_list = pd.read_csv('players_search_list.csv', header=None)

def firChar(name):
    return((str(name[0])).lower())

player_list [4] = list(map(lambda x: firChar(x) , player_list[0]))

#print(player_list)
type(player_list)
## sample url:
## https://www.basketball-reference.com/players/a/abdursh01.html

base_url = "https://www.basketball-reference.com/players"
row, col = player_list.shape
url = []

for i in range(row):
    player_chr = (player_list[4][i])
    player_name = (player_list[0][i])
    url.append(os.path.join(base_url,player_chr , player_name + ".html"))
player_list[5] = (url)

In [3]:
# chrome_options.add_argument("--headless") # make the chrome invisible
chrome_options = Options()
chrome_options.add_argument("--window-size=100x10")
wd = webdriver.Chrome(options=chrome_options)

In [28]:
total = []
df = pd.DataFrame()
for i in range(row):

    ## request the url
    url = player_list[5][i]
    html = requests.get(url).content
    soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
    links = soup.find_all('p')

    ## initialization
    m = "-"
    year = "-"
    pos = "-"
    coll = "-"
    draft = '-'
    foot = "-"

    ## process the info of the player
    try:
        fro , to = player_list[2][i].split('-')
        first, last = player_list[1][i].split()
    except:
        fro = player_list[2][i]
        to = '-'


    for link in links:
        tt = link.text
        tt = tt.split('\n')
        tt = list(map(lambda x: str.strip(x), tt))
        tt = list(filter(lambda x : x != "", tt))


        if "Born:" in tt :
            m = tt[1].split(',')[0]
            year = tt[2]
            try:
                month = str(datetime.strptime(m, '%B %d').month)
                day = str(datetime.strptime(m,'%B %d').day)
            except:
                month = '-'
                day = '-'
                print(m,year)
                pass

        elif "Position:" in tt:
            pos = tt[1]
            deli = ['and', '/']
            pos_str = ''
            s = False
            for i in range(len(deli)):
                if(deli[i] in pos):
                    s = True
                    pos = pos.split(deli[i])
            if s  == False:
                pos = [pos]
            pos = list(map(lambda x : ''.join(c for c in x if c.isupper()) , pos))
            pos = "-".join(pos)

        elif "College:" in tt:
            coll = tt[1]

        elif "Draft:" in tt:
            try:
                draft = tt[1]
                draft = re.findall('\d+',draft)[2]
            except:
                draft = '-'
                pass
            break

        ## Process the weight
        elif len(tt)> 0 and "(" in tt[0] and ")" in tt[0] and "-" in tt[0]:
            if foot != "-":
                continue

            try:
                foot, inches = (tt[0].split(',')[0]).split('-')
                pound =((tt[0].split("\xa0"))[1]).split("lb")[0]
            except:
                try:
                    pound =((tt[0].split(','))[1]).split("lb")[0]
                except:
                    pass

    if i % 500 == 0:
        print(i)


    appended_data = pd.Series([first, last, fro, to ,pos, foot, inches, pound, coll, month , day , year , pos  , draft ])  
    df = df.append(appended_data, ignore_index=True)
#     print('------------------------------------------------------------------------------------')

SyntaxError: 'break' outside loop (cell_name, line 5)

## Data Reprocess

In [29]:
clear_output()
break

f = df[5]
x = df[6]
p = df[7]

small_df = pd.DataFrame()

for i in range(0,row):
    if i % 100 == 0:
        print(i)
    try:
        junk= int(f[i])
        junk=int(x[i])
        junk=int(p[i])
    except:
        print('except', i,f[i],x[i],p[i])
        url = player_list[5][i]
        html = requests.get(url).content
        soup = BeautifulSoup(html,'html.parser',from_encoding='utf-8')
        links = soup.find_all('p')
        for link in links:
            tt = link.text
            tt = tt.split('\n')
            tt = list(map(lambda x: str.strip(x), tt))
            tt = list(filter(lambda x : x != "", tt))
            if len(tt)> 0 and "(" in tt[0] and ")" in tt[0] and "-" in tt[0] and 'lb' in tt[0]:
                try:
                    foot, inches = (tt[0].split(',')[0]).split('-')
                    pound =((tt[0].split("\xa0"))[1]).split("lb")[0]
                    print(i,foot,inches,pound)
                    print(tt)
                except:
                    try:
                        foot, inches = (tt[0].split(',')[0]).split('-')
                        pound =((tt[0].split(','))[1]).split("lb")[0]
                        print(i,foot,inches,pound)
                        print(tt)
                    except:
                        continue
            #appended_data = pd.Series([foot,inches,pound])  
            #small_df = small_df.append(appended_data, ignore_index=True)
            df[5][i] = foot
            df[6][i] = inches
            df[7][i] = pound


SyntaxError: 'break' outside loop (cell_name, line 5)

In [40]:
#appended_data = pd.Series([first, last, fro, to ,pos, foot, inches, pound, coll, month , day , year , pos  , draft ])
df = pd.read_excel('info.xlsx')
df.columns  = ['FirstName', 'LastName', 'From', 'To', '位置', '呎', '吋', '磅', 'Colleges','出生月','出生日', '出生年','位置', '選秀順位']